### Import libraries

We will be using BeautifulSoup, Pandas, and Altair to scrape, clean, and visualize the data respectively.

In [1]:
import re

import fandom
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
# Set the wiki root to search for
fandom.set_wiki("Valheim")
# Set the URL
url = "https://valheim.fandom.com/wiki/Food"

### Data preparation

It seems that the Valheim Wiki has compiled a list of food choices in the game. Good thing, pandas has a `read_html` function that allows me to get the table directly without having to write my own scraper. This function gives a list of DataFrame tables found in the given webpage.

In [3]:
df_list = pd.read_html(url, flavor="bs4")
# We only have 1 table in that page, so let's just get that one
df = df_list[0]

In [4]:
df.head(5)

,Name,Icon,Health,Stamina,Total,Healing (hp/tick),Duration (sec),Stack Size
0,Raspberries,NaN,7,20,27.0,1,600,50
1,Mushroom,NaN,15,15,30.0,1,900,50
2,Blueberries,NaN,8,25,33.0,1,600,50
3,Carrot,NaN,10,32,42.0,1,900,50
4,Cloudberries,NaN,13,40,53.0,1,900,50


It looks good, it seems that I have most of the information I need (Health, Stamina, Duration). The remaining column that I need is the stage/progression of the game. Perhaps I'll append it by other means. In addition, the `Icon` column are all NaNs. I can get them by scraping it manually:

In [5]:
content = requests.get(url)
soup = BeautifulSoup(content.text)
images_in_table = soup.find("table").find_all("a", class_="image", href=True)
image_links = [img["href"] for img in images_in_table]

# Replace Icon with the links
df["Icon"] = image_links

I might also need to get some other information from the infobox of individual pages. If that's the case then I can use this neat library called [fandom-py](https://github.com/NikolajDanger/fandom-py). It easily allows me to obtain the information that I need. There's some bit of data wrangling involved, so let's start rolling up our sleeves!

In [6]:
from typing import Dict


def get_infobox(row) -> Dict:
    """Get relevant information from the infobox"""
    name = row["Name"]
    url = ""
    drop_by_data = ""
    craft_data = ""

    try:
        # Get page and its URL
        page = fandom.page(name)
        url = page.url

        # Get infobox since we need a lot of info there
        infobox = page.content["infobox"]

        # Split the infobox for easier parsing
        # We just need to get 'Dropped by' and 'Crafting Materials'
        l = infobox.split("\n")
        if "Dropped by" in l:
            drop_idx = l.index("Dropped by")
            drop_by_data = l[drop_idx + 1]
        if "Crafting Materials" in l:
            craft_idx = l.index("Crafting Materials")
            craft_data_ = l[craft_idx + 1]
            craft_data = re.sub(
                r"(?<=\d)(?=[^\d\s])|(?<=[^\d\s])(?=\d)", " ", craft_data_
            )
    except fandom.error.PageError:
        print(f"Cannot find page for {name}, will return empty strings")

    return {"URL": url, "DroppedBy": drop_by_data, "Crafting": craft_data}

What I did is that I obtained the Fandom page for each food item, then filtered it with just the infobox. However, the return value is just one long string delimited by `\n`, so my workaround is to split it with that value. 

To get the key value pairs, I just took the *next* item of a specific element, assuming that's how it works. For example, to get the value for the key "Dropped by," I looked at the element after it. It seems to work!

Let's now put the URL, Drop information, and Craft information in one Dataframe and concatenate it with the rest of the crew!

In [42]:
infobox = pd.DataFrame([get_infobox(row) for _, row in df.iterrows()])

In [45]:
infobox.head()

,URL,DroppedBy,Crafting
0,https://valheim.fandom.com/en/wiki/Raspberries,Raspberry bushes in Meadows,
1,https://valheim.fandom.com/en/wiki/Mushroom,"Meadows, Black Forest, Swamp",
2,https://valheim.fandom.com/en/wiki/Blueberries,Blueberry bushes in the Black Forest biome,
3,https://valheim.fandom.com/en/wiki/Carrot,Farming Carrot seeds,
4,https://valheim.fandom.com/en/wiki/Cloudberries,Plains biome,


In [44]:
# Combine parsed infobox to our original dataframe
master_df = pd.concat([df, infobox], axis=1)
master_df.head()

,Name,Icon,Health,Stamina,Total,Healing (hp/tick),Duration (sec),Stack Size,URL,DroppedBy,Crafting,Progression,Duration (min),URL,DroppedBy,Crafting
0,Raspberries,https://static.wikia.nocookie.net/valheim/imag...,7.0,20.0,27.0,1,600,50,https://valheim.fandom.com/en/wiki/Raspberries,Raspberry bushes in Meadows,NaN,Stone,10.0,https://valheim.fandom.com/en/wiki/Raspberries,Raspberry bushes in Meadows,
1,Mushroom,https://static.wikia.nocookie.net/valheim/imag...,15.0,15.0,30.0,1,900,50,https://valheim.fandom.com/en/wiki/Mushroom,"Meadows, Black Forest, Swamp",NaN,Stone,15.0,https://valheim.fandom.com/en/wiki/Mushroom,"Meadows, Black Forest, Swamp",
2,Blueberries,https://static.wikia.nocookie.net/valheim/imag...,8.0,25.0,33.0,1,600,50,https://valheim.fandom.com/en/wiki/Blueberries,Blueberry bushes in the Black Forest biome,NaN,Stone,10.0,https://valheim.fandom.com/en/wiki/Blueberries,Blueberry bushes in the Black Forest biome,
3,Carrot,https://static.wikia.nocookie.net/valheim/imag...,10.0,32.0,42.0,1,900,50,https://valheim.fandom.com/en/wiki/Carrot,Farming Carrot seeds,NaN,Bronze,15.0,https://valheim.fandom.com/en/wiki/Carrot,Farming Carrot seeds,
4,Cloudberries,https://static.wikia.nocookie.net/valheim/imag...,13.0,40.0,53.0,1,900,50,https://valheim.fandom.com/en/wiki/Cloudberries,Plains biome,NaN,Silver,15.0,https://valheim.fandom.com/en/wiki/Cloudberries,Plains biome,


### Visualization

First, let's **remove Bukeperries because it's a special item**. Instead of giving you HP and Stamina, Bukeperries removes all the food you've ingested at the moment. 

In [8]:
master_df = master_df[df.Name != "Bukeperries"]

Then, let's typecast a few columns to visualize them properly:

In [9]:
master_df["Health"] = master_df["Health"].astype(float)
master_df["Stamina"] = master_df["Stamina"].astype(float)

Let's save the dataset into a CSV file for later use:

In [10]:
master_df.to_csv("data/Valheim_Food_Dataset.csv", index=False)

Now, it's time to plot! We'll use a scatterplot to see the distribution of HP and Stamina across food items. I'll arrange the plot in the following manner:
- X-axis will be the Stamina
- Y-axis will be the HP
- The size of the circle will represent the duration.

I will be using [Altair](https://altair-viz.github.io/) as my plotter of choice. It uses a declarative approach and seems more intuitive than [matplotlib](https://matplotlib.org/). 

In [11]:
import altair as alt

In [40]:
alt.Chart(df, title="Valheim Food Distribution", width=560, background="#fffff8").mark_circle().encode(
    alt.X("Stamina", scale=alt.Scale(zero=False)),
    alt.Y("Health", scale=alt.Scale(zero=False)),
    size="Duration (sec)",
    tooltip=[
        "Name",
        "DroppedBy",
        "Health",
        "Stamina",
        "Duration (sec)",
        "Healing (hp/tick)",
        "Crafting",
        "URL",
    ],
).interactive()

alt.Chart(...)

I'd like to separate each food category into its own "age" (i.e., stone, bronze, iron, and silver) because it would be nice if I can see what food items are available at my current level. 

However, it seems that I cannot find any table nor dataset that already has that. So I'll just manually annotate it by my own.

It's just 33 rows so it shouldn't be that tedious. My process is:
- Check the crafting requirement and its ore dependencies. [Deer stew](https://valheim.fandom.com/wiki/Deer_stew) is a Bronze age food because it requires [Cauldron](https://valheim.fandom.com/wiki/Cauldron), and a Cauldron is crafted using [Bronze](https://valheim.fandom.com/wiki/Bronze) ores.
- Check the recipe of each food. [Fish wrap](https://valheim.fandom.com/wiki/Fish_wraps) is an Iron age food because it requires [Barley Flour](https://valheim.fandom.com/wiki/Barley_flour) from a [Windmill](https://valheim.fandom.com/wiki/Windmill), and a Windmill is crafted using [Iron nails](https://valheim.fandom.com/wiki/Iron_nails).
- Lastly, I will also check the enemy types that you'll encounter to obtain a particular food. [Serpent stew](https://valheim.fandom.com/wiki/Serpent_stew) can be obtained from the [Serpent](https://valheim.fandom.com/wiki/Serpent) even during the Stone age (with a raft). However, killing the Serpent reliably requires Bronze age gear and up.

I will keep separate files for the annotated and unannotated dataset. I am only 100+ hours in the game, so if you see any discrepancies in my annotation, please let me know!

In [13]:
df = pd.read_csv("data/Valheim_Food_Dataset_Annotated.csv")

I'll update the Duration from seconds to minutes since that is the one displayed in the Valheim HUD.

In [14]:
df["Duration (min)"] = df["Duration (sec)"] / 60

In [41]:
selection = alt.selection_multi(fields=["Progression"], bind="legend")

alt.Chart(df, title="Valheim Food Distribution", width=560, background="#fffff8").mark_circle().encode(
    alt.X("Stamina", scale=alt.Scale(domain=(0, 80), zero=False)),
    alt.Y("Health", scale=alt.Scale(domain=(0, 90), zero=False)),
    size="Duration (min)",
    color=alt.Color(
        "Progression",
        scale=alt.Scale(
            scheme="tableau10", domain=["Silver", "Iron", "Bronze", "Stone"]
        ),
    ),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
    tooltip=[
        "Name",
        "DroppedBy",
        "Health",
        "Stamina",
        "Duration (min)",
        "Healing (hp/tick)",
        "Crafting",
        "URL",
    ],
).configure_axis(grid=False).add_selection(selection).interactive()

alt.Chart(...)